# Bias in word embeddings
Word embeddings is a technique in NLP and text mining to represent words in order to be able to compare words based on similarity. The word embeddings are trained on large data sets with text written by humans. Therefore, the bias we have and include in our writungs will be transferred to the word embeddings. This project looks at what bias there are in the swedish word embeddings "Swectors" based on text from Göteborgsposten. They will also be compared with another set of word ebedding that will be trained on antoher set of data.

In [1]:
import bz2
import pandas as pd
import numpy as np
import time

## Importing datasets
Importing the large set of vectors might take a little while, please be patient.

In [2]:
colnames = ["word"] + ["dim" + str(x) for x in range(1,301)]
with bz2.open("becctors-300dim.txt.bz2") as source:
    swectors = pd.read_csv(source, header=None, names=colnames, delimiter=" ", skiprows=[0])
    
swectors.tail()

,word,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,...,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299,dim300
130623,spoor,-0.305213,0.882732,0.522372,-0.725413,-0.925758,-0.059995,0.025186,-0.625011,-0.123028,...,0.151419,-0.101874,0.120103,-0.037013,0.021720,-0.205599,0.051005,0.069293,0.283596,-0.054317
130624,kuipers,0.211395,0.106868,0.075707,-0.720433,0.424596,0.285844,-0.178588,0.359769,0.243199,...,-0.638234,0.077303,-0.163403,0.530155,-0.298784,-0.163321,-0.170144,0.338389,-0.093671,0.033235
130625,ssewankambo,0.552984,-0.839261,1.116188,-0.220800,0.398118,-0.007519,-0.108604,0.092266,0.075665,...,0.004948,-0.537718,-0.013840,0.204205,0.051245,0.305431,-0.573084,0.451087,0.687805,0.167441
130626,efimova,0.106571,-0.482280,0.303211,0.781869,-0.760114,1.079618,-0.198003,0.797196,-0.437269,...,0.204036,-0.340340,0.587647,0.163961,0.016107,0.106837,-0.489778,-0.786732,-0.314826,-0.114501
130627,meraf,0.524146,-0.709905,0.344497,0.015716,-0.158800,0.649125,-0.070420,-0.098566,0.192166,...,0.199136,-0.264218,0.626736,0.190964,0.358741,-0.126234,-0.424287,-0.336282,-0.001072,0.152007


### Import small sized dataset

In [3]:
colnames = ["word"] + ["dim" + str(x) for x in range(1,301)]
with bz2.open("swectors_short-300dim.txt.bz2") as source:
    swectors_short = pd.read_csv(source, header=None, names=colnames, delimiter=" ", skiprows=[0])
    
swectors_short.tail()

,word,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,...,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299,dim300
7592,morötter,1.377063,-0.320867,-0.891954,2.830373,0.825997,1.932514,-2.163011,1.587833,1.440585,...,0.179074,0.170220,-2.506192,-2.462804,0.740179,0.003449,2.325763,1.437010,0.303144,0.385211
7593,hjältar,2.921456,-0.467303,-0.351971,-1.563029,2.213287,2.744328,0.347316,2.747632,-0.504926,...,-3.084410,-1.780909,-1.892354,0.929151,-1.926698,1.750462,-0.188903,-0.364937,0.745645,2.605387
7594,träff,0.857412,-1.905170,1.293573,1.165565,2.914954,0.528677,-1.657378,-1.054240,-1.095805,...,-2.051143,1.165688,-1.325770,-1.513558,-2.066850,-0.498036,1.108481,2.500199,-1.232332,0.314965
7595,varmare,0.044312,-3.489696,-3.128916,-0.886223,1.268917,-0.423831,1.517649,1.637759,1.587272,...,2.403448,2.708916,2.563368,0.494266,2.623131,-0.113758,-1.264208,0.773594,-1.736908,2.501356
7596,meddela,-2.005358,0.995493,-1.234387,-2.401334,-1.615172,-1.575130,2.422385,-1.812807,-0.771301,...,0.918420,-1.519924,0.996619,-2.035296,0.857632,2.701465,-0.780385,-0.184154,-3.955059,0.887238


Extract the vector for the word 'kvinna', in order to look att similar words for bias measures.
Get the 300 dimensions from the dataframe, convert to numpy and get the following format: `[[dim1 dim2 ... dim299 dim300]]`, take the first element to get a single list. Save it as a tuple with word first and vector second.

In [4]:
kvinna = ('kvinna', swectors.loc[swectors['word'] == 'kvinna'].loc[:, 'dim1':'dim300'].to_numpy()[0])
man = ('man', swectors.loc[swectors['word'] == 'man'].loc[:, 'dim1':'dim300'].to_numpy()[0])

In [5]:
def cosine_similarity(word1, word2):
    # Takes two vectors and calculates the cosine similarity between them
    # @ is dot product
    v1 = word1[1]
    v2 = word2[1]
    return (v1 @ v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))

In [6]:
cosine_similarity(kvinna, man)

0.1833739415716312

## Applying function to whole dataframe
This way of doing the calculation will apply a function to each row of the dataframe, and return a Series of same length with all results in it.

In [7]:
def n_most_similar(n, word, adj=False):
    
    if adj:
        df = swectors_filtered
        print("filtered")
    else:
        df = swectors
        print("not filtered")
        
    word_vec = (word, swectors.loc[swectors['word'] == word].loc[:, 'dim1':'dim300'].to_numpy()[0])
    
    def similarity(row):
        row_vec = (row['word'], row.loc['dim1':'dim300'].to_numpy())
        return cosine_similarity(word_vec, row_vec)

    start = time.time()
    similarities = df.apply(similarity, axis=1)

    # Concatenate the top n words (plus the word itself) to the similarity values of each word.
    # Also set the correct coulmn name.
    s1 = df.loc[similarities.nlargest(n+1).index, 'word']
    s2 = similarities.nlargest(n+1)
    similars = pd.concat([s1, s2], axis=1).rename(columns={0: "similarity"})
    end = time.time()
    print("Time elapsed: ", end - start)
    return similars

In [8]:
similars_kvinna = n_most_similar(10, 'kvinna')
print(similars_kvinna)

not filtered
Time elapsed:  85.86963391304016
           word  similarity
723      kvinna    1.000000
434        tjej    0.700042
10613   kvinnas    0.654458
3401        dam    0.632038
1247    kvinnan    0.631024
2210       brud    0.630095
405      person    0.582518
4211   kvinnlig    0.549536
9237    varelse    0.546948
450       kille    0.542245
720      flicka    0.531278


In [9]:
similars_män = n_most_similar(10, 'man')
print(similars_män)

not filtered
Time elapsed:  86.10235905647278
           word  similarity
12          man    1.000000
115        folk    0.470113
16593       mna    0.439708
0           det    0.412055
21           ju    0.403830
17           de    0.400845
789    personen    0.397858
13           du    0.396582
33          sig    0.373037
1598      denne    0.366336
45           ni    0.366264


## Filtering out all adjectives in the word embeddings
To see bias in adjectives, the dataframe with the swectors is filtered to only keep words that are in the dataframe with adjectves from Språkrådet.

In [10]:
with bz2.open("adjektiv.txt.bz2") as source:
    adjectives = pd.read_csv(source)
    
swectors_filtered = swectors.loc[swectors['word'].isin(adjectives['Word'])]

swectors_filtered.tail()

,word,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,...,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299,dim300
130198,opsykologiskt,0.826066,0.273885,-0.239991,0.012171,0.118731,-0.271874,-0.287778,0.574271,0.297365,...,-0.390943,-0.303843,-0.005978,0.457760,-0.350004,0.342862,-0.622444,0.694290,0.492464,0.260877
130217,felciterad,-0.370551,-0.078028,-0.508176,-0.408833,0.392929,0.889057,-0.341320,1.653109,-0.486973,...,-0.643841,0.098567,-0.372330,0.627091,0.352044,-0.231706,0.512707,0.019606,0.011067,0.651023
130301,reumatoid,-0.676586,-0.660290,-0.633994,-0.987734,-0.505051,-0.146731,-0.348949,-0.653860,0.243521,...,0.155815,-0.413890,0.915417,0.024261,-0.039707,-0.044997,-0.271445,-0.201023,0.348063,-1.119864
130369,stressrelaterade,-0.211277,-0.194241,0.048382,-0.087283,-0.988704,0.254020,0.100780,-0.209909,-0.787346,...,0.383015,0.123362,0.153442,-0.280903,0.424349,-0.702959,1.129862,0.148886,-0.024707,-0.078584
130503,finstrimlat,-0.235193,0.261199,-0.863190,-0.284448,0.175440,0.702909,-0.211578,-0.639635,-0.150278,...,0.178798,0.529680,0.416830,0.803025,-0.610747,0.182548,0.710888,-0.124730,0.496024,-0.011626


In [11]:
swectors_filtered.shape

(8500, 301)

In [17]:
similars_kvinna_adj = n_most_similar(10, 'tjej', adj=True)
print(similars_kvinna_adj)

filtered
Time elapsed:  4.297215461730957
                   word  similarity
19743      prostituerad    0.425860
4211           kvinnlig    0.425783
15893           lesbisk    0.402303
58027      transsexuell    0.385630
120645        kortvuxen    0.373026
100407       hivsmittad    0.369772
8369        homosexuell    0.348543
37146   rullstolsbunden    0.346129
54742        cancersjuk    0.337345
63871        kallblodig    0.335236
24911             tanig    0.330794
